<a href="https://colab.research.google.com/github/tj-ie/data_analytics/blob/main/get_geocoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
from collections import Counter
import time
from tqdm import tqdm
import csv


with open('./tmp/prized_teams.txt', 'r') as f:
  team = f.read().split("\n")
  team.remove('')

team_count = Counter(team)

team = []
cnt = []

for k, v in team_count.items():
  team.append(k)
  cnt.append(v)
  
# teams = dict(list(zip(team, cnt)))
# # keyで降順にsort
# sorted_teams = dict(sorted(teams.items(), key=lambda x: -x[1]))


url = 'http://www.geocoding.jp/api/'
global lats
global lons
lats = []
lons = []
for team_list in tqdm(team):
    payload = {'q': team_list}  # "v": 1.1,
    r = requests.get(url, params=payload)
    ret = BeautifulSoup(r.content, 'lxml')
    if ret.find('error'):
        raise ValueError(f"Invalid address submitted. {team_list}")
    else:
        lat = ret.find('lat').string
        lon = ret.find('lng').string
        lats.append(lat)
        lons.append(lon)
        time.sleep(10)

team_latlons = list(zip(team, lats, lons, cnt))

with open('./tmp/team_latlons_v3.csv', 'w') as f:
  w = csv.writer(f)
  for data_list in team_latlons:
    w.writerow(data_list)


In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import tqdm


URL = 'http://www.geocoding.jp/api/'


def coordinate(address):
    """
    addressに住所を指定すると緯度経度を返す。

    >>> coordinate('東京都文京区本郷7-3-1')
    ['35.712056', '139.762775']
    """
    payload = {'q': address}
    html = requests.get(URL, params=payload)
    soup = BeautifulSoup(html.content, "html.parser")
    if soup.find('error'):
        raise ValueError(f"Invalid address submitted. {address}")
    latitude = soup.find('lat').string
    longitude = soup.find('lng').string
    return [latitude, longitude]


def coordinates(addresses, interval=10, progress=True):
    """
    addressesに住所リストを指定すると、緯度経度リストを返す。

    >>> coordinates(['東京都文京区本郷7-3-1', '東京都文京区湯島３丁目３０−１'], progress=False)
    [['35.712056', '139.762775'], ['35.707771', '139.768205']]
    """
    coordinates = []
    for address in progress and tqdm(addresses) or addresses:
        coordinates.append(coordinate(address))
        time.sleep(interval)
    return coordinates